In [1]:
import re
from tqdm.notebook import tqdm
from datasets import load_dataset

In [2]:
from llama_index import VectorStoreIndex, StorageContext, SimpleDirectoryReader, download_loader, load_index_from_storage
from llama_index.schema import Document
from llama_index.tools import QueryEngineTool, ToolMetadata
from llama_index.agent import ContextRetrieverOpenAIAgent
from llama_index.prompts import PromptTemplate

In [3]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts.chat import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
    SystemMessagePromptTemplate,
)
from langchain.schema import HumanMessage, SystemMessage

## Define prompt template

In [4]:
system_template = (
    'Always answer as if you were a customer service agent. '
    'Prefer to use "we" instead of "they"/"them". '
    'Give simple and direct answers.'
)
human_template = "{text}"

chat_prompt = ChatPromptTemplate.from_messages([
    ("system", system_template),
    ("human", human_template),
])

In [24]:
lamma_index_template = (
    "We have provided context information below. \n"
    "---------------------\n"
    'Always answer as if you were a customer service agent. '
    'Prefer to use "we" instead of "they"/"them". '
    'Give simple and direct answers.'
    "\n---------------------\n"
    "Given this information, please answer the question: {query_str}\n"
)

## Define eval data

In [36]:
eval_data = [
    {
        "original_input": "My robotic arm seems to be misaligned. It's not picking objects accurately. What can I do?",
        "original_output": "It appears that the arm may need recalibration. Please follow the instructions in the user manual to reset the calibration settings. If the problem persists, feel free to contact us again.",
        "input": chat_prompt.format_messages(
            text="My robotic arm is missing the objects it's trying to pick up. Can you please help me?",
        ),
        "default_output": None,
        "fine_tuned_output": None,
        "rag_output": None,
    },
    {
        "original_input": "Our robotic arm's movements seem to be less precise than before.",
        "original_output": "Inconsistent precision might be caused by mechanical wear or environmental factors affecting sensors. Please calibrate the arm and ensure the workspace is clean and free from obstacles. If the problem continues, we can explore additional solutions to improve the arm's precision.",
        "input": chat_prompt.format_messages(
            text="It looks like the arm is losing precision as time goes on. How can I make this better?",
        ),
        "default_output": None,
        "fine_tuned_output": None,
        "rag_output": None,
    },
    {
        "original_input": "After a recent firmware update, the robotic arm is making unexpected movements.",
        "original_output": "Firmware updates can sometimes introduce new behavior. Check the release notes for any changes related to arm movements. If the movements are unintended, roll back to the previous firmware version and report the issue to us for investigation.",
        "input": chat_prompt.format_messages(
            text="My robotic arm has been weird since the last firmware update...",
        ),
        "default_output": None,
        "fine_tuned_output": None,
        "rag_output": None,
    },
    {
        "original_input": "The robotic arm struggles to perform tasks in low light conditions, affecting its operation during nighttime.",
        "original_output": "Low light conditions might impair sensor performance and object detection. Consider using additional lighting or upgrading the sensors for low light capabilities. We'll assist you in optimizing performance in low light conditions.",
        "input": chat_prompt.format_messages(
            text="Sometimes my robotic arm does not work as other times. I'm not sure, but it looks like it works better during the morning/afternoon than during the night. What could be causing this?"
        ),
        "default_output": None,
        "fine_tuned_output": None,
        "rag_output": None,
    },
    {
        "original_input": "The gripping force of the robotic arm's end effector is inconsistent, causing issues with handling fragile objects.",
        "original_output": "Inconsistent gripping force can be due to various factors, such as worn-out gripper components or inaccurate calibration. Check and maintain the gripper's mechanical parts, and recalibrate the gripping force if needed.",
        "input": chat_prompt.format_messages(
            text="The arm has broken a glass that it was trying to pick up. How can I make it grip with less force?"
        ),
        "default_output": None,
        "fine_tuned_output": None,
        "rag_output": None,
    },
    {
        "original_input": "Our robotic arm occasionally jerks suddenly during operation. What could be the problem?",
        "original_output": "Sudden jerky movements can result from motor control issues or improper synchronization. Check the motor control settings and ensure all axes move smoothly. If the issue continues, we'll help you diagnose and fix the problem.",
        "input": chat_prompt.format_messages(
            text="Sometimes my arm makes random movements/twitches - how can I fix this?"
        ),
        "default_output": None,
        "fine_tuned_output": None,
        "rag_output": None,
    },
    {
        "original_input": "-",
        "original_output": "-",
        "input": chat_prompt.format_messages(
            text="How tall is Barack Obama?"
        ),
        "default_output": None,
        "fine_tuned_output": None,
        "rag_output": None,
    },
    {
        "original_input": "-",
        "original_output": "-",
        "input": chat_prompt.format_messages(
            text="Is it possible to order a 3D printed substitution part from you guys?"
        ),
        "default_output": None,
        "fine_tuned_output": None,
        "rag_output": None,
    },
    {
        "original_input": "-",
        "original_output": "-",
        "input": chat_prompt.format_messages(
            text="How can I get substitution parts from you guys?"
        ),
        "default_output": None,
        "fine_tuned_output": None,
        "rag_output": None,
    },
]

## Load and process dataset

In [6]:
dataset = load_dataset("FunDialogues/customer-service-robot-support")

In [7]:
df = list(dataset.values())[0].to_pandas()

In [8]:
pattern = re.compile(r'(Customer:|Agent:)')
df['split_dialogue'] = df['dialogue'].apply(lambda x: [substring.strip() for substring in pattern.split(x)[1:]])
df['json_line'] = df['split_dialogue'].apply(
    lambda x: {
        "messages": [
            {"role": "system", "content": system_template},
            {"role": "user", "content": x[1]},
            {"role": "assistant", "content": x[3]},
        ]
    }
)

In [9]:
df.head()

,index,id,description,dialogue,split_dialogue,json_line
0,0,1,Robotic arm calibration issue,Customer: My robotic arm seems to be misaligne...,"[Customer:, My robotic arm seems to be misalig...","{'messages': [{'role': 'system', 'content': 'A..."
1,1,2,Overheating problem in production line arm,Customer: The robotic arm in our production li...,"[Customer:, The robotic arm in our production ...","{'messages': [{'role': 'system', 'content': 'A..."
2,2,3,Robotic arm making strange noises,Customer: Our robotic arm has been making a we...,"[Customer:, Our robotic arm has been making a ...","{'messages': [{'role': 'system', 'content': 'A..."
3,3,4,Sudden jerky movements in the robotic arm,Customer: Our robotic arm occasionally jerks s...,"[Customer:, Our robotic arm occasionally jerks...","{'messages': [{'role': 'system', 'content': 'A..."
4,4,4,Robotic arm not responding to commands,Customer: My robotic arm is not responding to ...,"[Customer:, My robotic arm is not responding t...","{'messages': [{'role': 'system', 'content': 'A..."


In [10]:
output_file = "llm_training_data.jsonl"
df['json_line'].to_json(output_file, orient='records', lines=True)

## Fine tuning

In [11]:
from openai import OpenAI

In [14]:
fine_tune = False

In [15]:
if fine_tune:
    client = OpenAI()
    
    fine_tuning_file = client.files.create(
      file=open(output_file, "rb"),
      purpose="fine-tune"
    )
    
    fine_tuning_job = client.fine_tuning.jobs.create(
      training_file=fine_tuning_file.id, 
      model="gpt-3.5-turbo"
    )

In [17]:
if fine_tune:
    fine_tuning_result = client.fine_tuning.jobs.retrieve(fine_tuning_job.id)
    fine_tuned_model_name = fine_tuning_result.fine_tuned_model
    if fine_tuning_result.fine_tuned_model:
        print("Fine-tuning process completed.")
        print(f"New model name: {fine_tuned_model_name}")
    else:
        print("Fine-tuning process still ongoing.")
else:
    print("Will use previously existing fine-tuned model.")
    fine_tuned_model_name = "ft:gpt-3.5-turbo-0613:personal::8cdLRUzu"

Will use previously existing fine-tuned model.


## RAG Definition

In [18]:
index = VectorStoreIndex.from_documents([Document(text=doc) for doc in df["dialogue"].to_list()])
index.storage_context.persist(persist_dir="./storage/customer-service-robot-support")
query_engine = index.as_query_engine()

In [19]:
context_agent = ContextRetrieverOpenAIAgent.from_tools_and_retriever(
    [],
    index.as_retriever(similarity_top_k=3),
    verbose=False,
)

## Eval the models

In [20]:
default_model = ChatOpenAI(temperature=0)

fine_tuned_model = ChatOpenAI(
    temperature=0, model_name=fine_tuned_model_name
)

rag_model = context_agent

In [37]:
for i, chat in enumerate(eval_data):
    print(f"Working on eval sample #{i}...")
    chat["default_output"] = default_model(
        chat_prompt.format_messages(
            text=chat["input"]
        )
    )
    chat["fine_tuned_output"] = fine_tuned_model(
        chat_prompt.format_messages(
            text=chat["input"]
        )
    )
    chat["rag_output"] = rag_model.chat(
        PromptTemplate(lamma_index_template).format(query_str=chat["input"][1].content)
    ).response

Working on eval sample #0...
Working on eval sample #1...
Working on eval sample #2...
Working on eval sample #3...
Working on eval sample #4...
Working on eval sample #5...
Working on eval sample #6...
Working on eval sample #7...
Working on eval sample #8...


In [38]:
for i, chat in enumerate(eval_data):
    print(f'Chat #{i}')
    print(f'\tOriginal Input:\n\t\t{chat["original_input"]}\n\n')
    print(f'\tOriginal Output:\n\t\t{chat["original_output"]}\n\n')
    print(f'\tNew Input:\n\t\t{chat["input"][1].content}\n\n')
    print(f'\tDefault Output:\n\t\t{chat["default_output"].content}\n\n')
    print(f'\tFine-tuned Output:\n\t\t{chat["fine_tuned_output"].content}\n\n')
    print(f'\tRAG Output:\n\t\t{chat["rag_output"]}')
    
    print('\n\n----------------------------------------------------------------------------------------------------\n\n')

Chat #0
	Original Input:
		My robotic arm seems to be misaligned. It's not picking objects accurately. What can I do?


	Original Output:
		It appears that the arm may need recalibration. Please follow the instructions in the user manual to reset the calibration settings. If the problem persists, feel free to contact us again.


	New Input:
		My robotic arm is missing the objects it's trying to pick up. Can you please help me?


	Default Output:
		Of course, we're here to help! If your robotic arm is having trouble picking up objects, there are a few things you can try. First, make sure that the arm is properly calibrated and aligned. You may also want to check if there are any obstructions or debris that could be interfering with its operation. If the issue persists, please provide us with more details so we can assist you further.


	Fine-tuned Output:
		Object detection issues can occur due to sensor calibration or environmental factors. Check the sensor's calibration and ensure the